In [1]:
using MNIST
trainX, trainY = traindata()
testX, testY = testdata()
trainY=Int.(trainY)
testY=Int.(testY)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{Float64}, ::Int64, ::Int64) at .\deprecated.jl:57
 [3] traindata() at C:\Users\User\AppData\Local\JuliaPro-0.6.0.1\pkgs-0.6.0.1\v0.6\MNIST\src\MNIST.jl:88
 [4] include_string(::String, ::String) at .\loading.jl:515
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\User\AppData\Local\JuliaPro-0.6.0.1\pkgs-0.6.0.1\v0.6\IJulia\src\execute_request.jl:160
 [6] eventloop(::ZMQ.Socket) at C:\Users\User\AppData\Local\JuliaPro-0.6.0.1\pkgs-0.6.0.1\v0.6\IJulia\src\eventloop.jl:8
 [7] (::IJulia.##11#14)() at .\task.jl:335
while loading In[1], in expression starting on line 2
Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{Float64}, ::Int64) at .\deprecated.jl:57
 [3] traindata() at C:\Users\User\AppData\Local\JuliaPro-0.6.0.1\pkgs-0.6.0.1\v0.6\MNIST\src\MNIST.jl:89
 [4] include_string(::String, ::String) at .\loading.jl:515
 [5] execute_request(::ZMQ.Socket, ::IJuli

10000-element Array{Int64,1}:
 7
 2
 1
 0
 4
 1
 4
 9
 5
 9
 0
 6
 9
 ⋮
 5
 6
 7
 8
 9
 0
 1
 2
 3
 4
 5
 6

In [2]:
type NeuralNetwork
    numOfLayers::Int
    sizes::Array{Int,1}
    biases::Array{Array{Float64,1},1}
    weights::Array{Array{Float64,2},1}
    function NeuralNetwork(s)
        numOfLayers=length(s)
        sizes=s
        biases=[rand(y) for y in s[2:end]]
        weights=[rand(y,x) for (x,y) in zip(s[1:end-1],s[2:end])]
        new(numOfLayers,sizes,biases,weights)
    #return biases, weights
    end
end

In [3]:
net = NeuralNetwork([784,15,10])

NeuralNetwork(3, [784, 15, 10], Array{Float64,1}[[0.406031, 0.381599, 0.180761, 0.900402, 0.259479, 0.582121, 0.756559, 0.168323, 0.208358, 0.560963, 0.906883, 0.502223, 0.889426, 0.396306, 0.598851], [0.941026, 0.535058, 0.278317, 0.318891, 0.531942, 0.955771, 0.0228666, 0.0190559, 0.519874, 0.779196]], Array{Float64,2}[[0.867118 0.893728 … 0.169862 0.298852; 0.944274 0.766854 … 0.507048 0.869068; … ; 0.200642 0.00792705 … 0.432287 0.583057; 0.45196 0.718425 … 0.785632 0.857203], [0.75614 0.109277 … 0.34928 0.032948; 0.743618 0.792786 … 0.61498 0.562394; … ; 0.984853 0.221298 … 0.136003 0.653604; 0.884174 0.735888 … 0.157874 0.148666]])

In [4]:
function sigmoid(z)
  return 1.0 / (1.0 + exp(-z))
end


sigmoid (generic function with 1 method)

In [5]:
function feedforward(net,a)
    for (b,w) in zip(net.biases, net.weights)
        a = sigmoid.((w*a)+b)
    end
    return a
end


feedforward (generic function with 1 method)

In [6]:
function SGD(net, training_data, epochs, mini_batch_size, eta,test_data)
    n_test = length(test_data)
    n = length(training_data)
    training_data2=collect(training_data)
    mini_batches = [training_data2[k:k+(mini_batch_size-1)] for k=1:mini_batch_size:n-1]
    for j in 1:epochs
        for mini_batch in mini_batches
            update_mini_batch(net, mini_batch, eta)
        end
       
        print("epoch ",j, ": ",evaluate(net,test_data),"  ")
        
    end
end

SGD (generic function with 1 method)

In [7]:
function update_mini_batch(net, mini_batch, eta)
    #def fn(a, y):
     #   return np.sum(np.nan_to_num(-y*log(a)-(1-y)*log(1-a)))
    nabla_b = [zeros(size(b)) for b in net.biases]
    nabla_w = [zeros(size(w)) for w in net.weights]
    for (x, y) in mini_batch
        delta_nabla_b, delta_nabla_w = backprop(net,x, y)
        nabla_b = [nb+dnb for (nb, dnb) in zip(nabla_b, delta_nabla_b)]
        nabla_w = [nw+dnw for (nw, dnw) in zip(nabla_w, delta_nabla_w)]
    end
    net.weights = [w-(eta/size(mini_batch,1))*nw for (w, nw) in zip(net.weights, nabla_w)]
    net.biases = [b-(eta/size(mini_batch,1))*nb for (b, nb) in zip(net.biases, nabla_b)]
    
    return net.weights,net.biases
end

update_mini_batch (generic function with 1 method)

In [8]:
function backprop(net,x, y)
    nabla_b = [zeros(size(b)) for b in net.biases]
    nabla_w = [zeros(size(w)) for w in net.weights]
    # feedforward
    activation = x
    activations = [x] # list to store all the activations, layer by layer
    zs = [] # list to store all the z vectors, layer by layer
    for (b, w) in zip(net.biases, net.weights)
        #z = dot(w, activation)+b
        z = (w*activation)+b
        push!(zs,z)
        activation = sigmoid.(z)
        push!(activations,activation)
    end
    # backward pass
    delta1 = Array(cost_derivative(net,activations[end], y) .* sigmoid_prime.(zs[end]))
    delta=[]
    for i in delta1
        push!(delta,i)
    end
    nabla_b[end] = delta
    nabla_w[end] = delta*transpose(activations[end-1])
    # Note that the variable l in the loop below is used a little
    # differently to the notation in Chapter 2 of the book.  Here,
    # l = 1 means the last layer of neurons, l = 2 is the
    # second-last layer, and so on.  It's a renumbering of the
    # scheme in the book, used here to take advantage of the fact
    # that Python can use negative indices in lists.
    for l=net.numOfLayers-1:-1:2
        z = zs[l-1]
        #println(size(net.weights[l]), " ", size(delta)," ", size(sp))
        delta = (transpose(net.weights[l])*delta) 
        nabla_b[l-1] = delta
        nabla_w[l-1] = (delta * transpose(activations[l-1]))
    end
    return nabla_b, nabla_w
end

backprop (generic function with 1 method)

In [9]:
function evaluate(net,test_data)
    test_results = [(findmax(feedforward(net,x))[2], findmax(y)[2]) for (x, y) in test_data]
    #println(test_results)
    return sum((x == y) for (x, y) in test_results) / size(test_data)[1]
end

evaluate (generic function with 1 method)

In [10]:
function cost_derivative(net,output_activations, y)
    return output_activations-y
end

cost_derivative (generic function with 1 method)

In [11]:
function sigmoid_prime(z)
     return sigmoid(z)*(1-sigmoid(z))
end

sigmoid_prime (generic function with 1 method)

In [12]:
function vectorized_result(j)
    e = zeros(10, 1)
    e[j+1] = 1.0
    return e
end

vectorized_result (generic function with 1 method)

In [13]:
training_inp=reshape(trainX,(60000,784))
training_inputs=[]
for i=1:size(training_inp)[1]
    push!(training_inputs,training_inp[i,:])
end
training_outputs = [vectorized_result(y) for y in trainY]
training_data = zip(training_inputs, training_outputs)

test_inp=reshape(testX,(10000,784))
test_inputs=[]
for i=1:size(test_inp)[1]
    push!(test_inputs,test_inp[i,:])
end
test_outputs = [vectorized_result(y) for y in testY]
test_data = zip(test_inputs, test_outputs)


Base.Iterators.Zip2{Array{Any,1},Array{Array{Float64,2},1}}(Any[[0.0, 0.0, 73.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 194.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 249.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 208.0, 0.0], [0.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  41.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 0.0], [0.0, 92.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  132.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 67.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  213.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  254.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  253.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 254.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 1

In [ ]:
SGD(net,training_data, 100 , 30 , 3.0, test_data)

epoch 1: 0.1028  epoch 2: 0.1028  epoch 3: 0.1028  epoch 4: 0.1028  epoch 5: 0.1028  epoch 6: 0.1028  epoch 7: 0.1028  epoch 8: 0.1028  epoch 9: 0.1028  epoch 10: 0.1028  epoch 11: 0.1028  epoch 12: 0.1028  epoch 13: 0.1028  epoch 14: 0.1028  epoch 15: 0.1028  epoch 16: 0.1028  epoch 17: 0.1028  epoch 18: 0.1028  epoch 19: 0.1027  epoch 20: 0.1028  epoch 21: 0.1029  epoch 22: 0.103  epoch 23: 0.1031  epoch 24: 0.103  epoch 25: 0.103  epoch 26: 0.103  epoch 27: 0.1033  epoch 28: 0.1035  epoch 29: 0.1039  epoch 30: 0.1038  epoch 31: 0.1039  epoch 32: 0.1036  epoch 33: 0.1035  epoch 34: 0.1024  epoch 35: 0.1024  epoch 36: 0.102  